In [2]:
import numpy as np
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

W = np.array([
    [5, 5, -5],
    [5, 5, -5],
    [-5, -5, 5]
])
1
x = {}

n,m = W.shape
for i in range(n):
    for j in range(m):
        x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))

        
COSTS = [W[i][j] * x[i,j] for i in range(n)\
       for j in range(m)]
solver.Maximize(solver.Sum(COSTS))


sol = solver.Solve()

print('Total cost = ', solver.Objective().Value())

F = ['a', 'b', 'c']
S = ['d', 'e', 'f']

for i in range(n):
    for j in range(m):
        v = x[i,j].solution_value()
        print(F[i] + '-' + S[j] + " => " + str(int(v)))

Total cost =  25.0
a-d => 1
a-e => 1
a-f => 0
b-d => 1
b-e => 1
b-f => 0
c-d => 0
c-e => 0
c-f => 1


# graph cut for limbs

Assuming two sets of joints $J_1, J_2$ that belong to a limb, with $\vert J_1 \vert = n, \vert J_2 \vert = m$.
The graph consists of two sets of edges:

* limb edge: $\lambda \in \{0,1 \}^{\vert n \times m \vert}$
* joint edge: $\iota \in \{ 0,1\}^{n\times n + m \times m}$

We want to minimize the following cost function:

$$
E = \biggr(\sum_{j_1}^{J_1} \sum_{j_2}^{J_2} \lambda(j_1, j_2) \Phi(j_1, j_2) \biggr)
+ \biggr(\sum_{j_1}^{J_1} \sum_{j_2}^{J_1} \iota(j_1, j_2)\Pi(j_1, j_2) \biggr)
+ \biggr(\sum_{j_1}^{J_2} \sum_{j_2}^{J_2} \iota(j_1, j_2)\Pi(j_1, j_2) \biggr)
$$

given the following transitivity conditions:

$$\lambda(a,c) + \lambda(b,c) - 1 \leq \iota(a,b)$$
$$\lambda(a,c) + \iota(a,b) - 1 \leq \lambda(b,c)$$

In [15]:
import numpy as np
L = np.array([
    [5, 5, 0, 0.5],
    [5, 6, -1, -5],
    [-1,0, 1, 5],
    [-5,0, 5, 3]
])


# J1, J2 symmetric
J1 = np.array([
    [0, 5, 1, 1],
    [5, 0, 0, 0],
    [1, 0, 0, 5],
    [1, 0, 5, 0]
])

J2 = np.array([
    [0, 5, 1, 1],
    [5, 0, 0, 0],
    [1, 0, 0, 5],
    [1, 0, 5, 0]
])


def solve_graph(L, J1, J2):

    solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                               pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    n, m = L.shape

    lm = {}
    for i in range(n):
        for j in range(m):
            lm[i,j] = solver.BoolVar('lambda[%i,%i]' % (i,j))

    io1 = {}
    for i in range(n):
        for j in range(n):
            io1[i,j] = solver.BoolVar('iota1[%i,%i]' % (i,j))

    io2 = {}
    for i in range(m):
        for j in range(m):
            io2[i,j] = solver.BoolVar('iota2[%i,%i]' % (i,j))

    for a in range(n):
        for b in range(n):
            for c in range(m):
                solver.Add(lm[a,c] + lm[b,c] -1 <= io1[a,b])
                solver.Add(lm[a,c] + io1[a,b] - 1 <= lm[b,c])

    for a in range(m):
        for b in range(m):
            for c in range(n):
                solver.Add(lm[a,c] + lm[b,c] -1 <= io2[a,b])
                solver.Add(lm[a,c] + io2[a,b] - 1 <= lm[b,c])

    # first objective function
    sum1 = solver.Sum(lm[a,b] * L[a,b] for a in range(n) for b in range(m))
    sum2 = solver.Sum(io1[a,b] * J1[a,b] for a in range(n) for b in range(n))
    sum3 = solver.Sum(io2[a,b] * J2[a,b] for a in range(m) for b in range(m))

    solver.Maximize(sum1 + sum2 + sum3)

    # -- solver --
    solver.Solve()
    print("Time = ", solver.WallTime(), " ms")

    L_graph = []
    J1_graph = []
    J2_graph = []
    
    for a in range(n):
        for b in range(m):
            if lm[a,b].solution_value() > 0:
                L_graph.append((a,b))
    
    for a in range(n):
        for b in range(n):
            if io1[a,b].solution_value() > 0:
                J1_graph.append((a,b))
    
    for a in range(m):
        for b in range(m):
            if io2[a,b].solution_value() > 0:
                J2_graph.append((a,b))
    
    return L_graph, J1_graph, J2_graph

Lg, J1g, J2g = solve_graph(L, J1, J2)

all constraints are set
Time =  15  ms
